In [1]:
import tensorflow as tf
import numpy as np
import input_data

In [2]:
batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

# The shape of the output tensor. 
# The standard deviation of the normal distribution.
# tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
# https://www.tensorflow.org/versions/r0.10/api_docs/python/constant_op/random_tensors#random_normal

In [3]:
def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,
                                 strides=[1, 1, 1, 1], padding='SAME')) # l1a shape 28x28x1 
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1], 
                       strides=[1, 2, 2, 1], padding='SAME') # l1 shape 14x14x32
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,
                                 strides=[1, 1, 1, 1], padding='SAME')) # l2a shape 14x14x64
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1], 
                       strides=[1, 2, 2, 1], padding='SAME') # l2 shape 7x7x64
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,
                                 strides=[1, 1, 1, 1], padding='SAME')) # l3a shape 7x7x128
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],
                       strides=[1, 2, 2, 1], padding='SAME') # l3 shape 4x4x128
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
    #  w4.get_shape().as_list()[0] == 1x2048
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    
    pyx = tf.matmul(l4, w_o)
    return pyx



In [4]:
# MNIST_data를 받아 옵니다. (한번만 실행하면 됩니다.)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)
teX = teX.reshape(-1, 28, 28, 1)

print('trx', len(trX))
print('tex', len(teX))

Extracting MNIST_data/train-images-idx3-ubyte.gz


C:\Users\minho_kim\Anaconda3\lib\gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
C:\Users\minho_kim\Downloads\ML\CNN\input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
trx 55000
tex 10000


In [5]:
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs     
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # 128의 필터에 4 * 4 이미지
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

print(X)
print(w)
print(w2)
print(w3)
print(w4)
print(w_o)

Output("Placeholder:0", shape=(?, 28, 28, 1), dtype=float32)
Output("Variable/read:0", shape=(3, 3, 1, 32), dtype=float32)
Output("Variable_1/read:0", shape=(3, 3, 32, 64), dtype=float32)
Output("Variable_2/read:0", shape=(3, 3, 64, 128), dtype=float32)
Output("Variable_3/read:0", shape=(2048, 625), dtype=float32)
Output("Variable_4/read:0", shape=(625, 10), dtype=float32)


In [7]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for i in range(3):
        training_batch = zip(range(0, len(trX), batch_size), #len(trX) == 55000, batch_size == 128
                            range(batch_size, len(trX), batch_size))
        
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                         p_keep_conv: 0.8, p_keep_hidden: 0.5})
            test_indices = np.arange(len(teX)) #len(teX) == 10000
            np.random.shuffle(test_indices)
            test_indices = test_indices[0:test_size] # test_indices = [random한 128개]
            print(sess.run(cost, feed_dict={X: trX[start:end], Y: trY[start:end], p_keep_conv:0.8, p_keep_hidden: 0.5}))
            print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                            sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                           Y: teY[test_indices],
                                                           p_keep_conv: 1.0,
                                                           p_keep_hidden: 1.0}))) # axis =1 rows 기준 즉, 128개 중에서 최고인 teY 뽑아라.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
2.30258
0 0.13671875
2.30258
0 0.140625
2.30258
0 0.171875
2.30257
0 0.125
2.30258
0 0.12890625
2.30258
0 0.12890625
2.30257
0 0.11328125
2.30258
0 0.15234375
2.30258
0 0.1484375
2.30258
0 0.12109375
2.30258
0 0.12890625
2.30258
0 0.13671875
2.30259
0 0.109375
2.30258
0 0.1640625
2.30258
0 0.171875
2.30259
0 0.15625
2.30257
0 0.140625
2.30258
0 0.140625
2.30258
0 0.16015625
2.30258
0 0.15234375
2.30258
0 0.1171875
2.30258
0 0.15625
2.30258
0 0.10546875
2.30258
0 0.1328125
2.30258
0 0.1328125
2.30258
0 0.1015625
2.30258
0 0.125
2.30258
0 0.1328125
2.30259
0 0.1484375
2.30258
0 0.12890625
2.30259
0 0.11328125
2.30259
0 0.125
2.30258
0 0.12890625
2.30258
0 0.14453125
2.30258
0 0.171875
2.30259
0 0.125
2.30258
0 0.140625
2.30258
0 0.125
2.30259
0 0.1328125
2.30257
0 0.15625
2.30258
0 0.140625
2.30259
0 0.19921875
2.30258
0 0.12890625
2.30259
0 0.1484375
2.30258
0 0.1796875
2.30258
0 0.109375
2.30258
0 0.13671875
2.30